In [27]:
import tarfile
import pandas as pd
import io
import scanpy as sc
import os
import numpy as np
import anndata as ad

In [16]:
# Define the full path to the .tar file on the server
tar_file_path = '/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_RAW.tar'

In [17]:
# Open the .tar file in read mode
try:
    with tarfile.open(tar_file_path, 'r') as tar:
        # Get a list of all members (files and folders) inside the archive
        contents = tar.getmembers()
        
        # Print the names of the files
        print("Contents of the .tar file:")
        for member in contents:
            print(member.name)
            
except FileNotFoundError:
    print(f"Error: The file was not found at {tar_file_path}. Please check the path.")
except tarfile.ReadError:
    print(f"Error: Could not read the file at {tar_file_path} as a .tar archive.")

Contents of the .tar file:
GSM7473682_HC_a_exprMat_file.csv.gz
GSM7473682_HC_a_fov_positions_file.csv.gz
GSM7473682_HC_a_metadata_file.csv.gz
GSM7473682_HC_a_tx_file.csv.gz
GSM7473682_HCa.tar.gz
GSM7473682_HCa_CellLabels.tar.gz
GSM7473682_HCa_CompartmentLabels.tar.gz
GSM7473683_HC_b_exprMat_file.csv.gz
GSM7473683_HC_b_fov_positions_file.csv.gz
GSM7473683_HC_b_metadata_file.csv.gz
GSM7473683_HC_b_tx_file.csv.gz
GSM7473683_HCb.tar.gz
GSM7473683_HCb_CellLabels.tar.gz
GSM7473683_HCb_CompartmentLabels.tar.gz
GSM7473684_HC_c_exprMat_file.csv.gz
GSM7473684_HC_c_fov_positions_file.csv.gz
GSM7473684_HC_c_metadata_file.csv.gz
GSM7473684_HC_c_tx_file.csv.gz
GSM7473684_HCc.tar.gz
GSM7473684_HCc_CellLabels.tar.gz
GSM7473684_HCc_CompartmentLabels.tar.gz
GSM7473685_UC_a_exprMat_file.csv.gz
GSM7473685_UC_a_fov_positions_file.csv.gz
GSM7473685_UC_a_metadata_file.csv.gz
GSM7473685_UC_a_tx_file.csv.gz
GSM7473685_UCa.tar.gz
GSM7473685_UCa_CellLabels.tar.gz
GSM7473685_UCa_CompartmentLabels.tar.gz
GSM747368

In [7]:
# The name of the specific file inside the tar archive you want to check
file_to_check = 'GSM7473682_HC_a_fov_positions_file.csv.gz'

try:
    with tarfile.open(tar_file_path, 'r') as tar:
        # Get the member (file) from the tar archive
        member = tar.getmember(file_to_check)
        
        # Extract the file object from the tar archive
        file_obj = tar.extractfile(member)
        
        if file_obj is not None:
            # Read the gzipped CSV file directly into a pandas DataFrame
            df = pd.read_csv(io.BytesIO(file_obj.read()), compression='gzip')
            
            # Display the first 30 rows
            print(f"Contents of {file_to_check}:")
            print(df.head(30))
        else:
            print(f"Error: Could not find '{file_to_check}' inside the archive.")
            
except tarfile.ReadError:
    print("Error: The file could not be read as a tar archive.")
except FileNotFoundError:
    print(f"Error: The main archive '{tar_file_path}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Contents of GSM7473682_HC_a_fov_positions_file.csv.gz:
    fov   x_global_px    y_global_px
0     1  17155.555556  169755.555556
1     2  11611.111111  169544.444444
2     3   6100.000000  168905.555556
3     4  -1961.111111  167177.777778
4     5  -5255.555556  163488.888889
5     6  -7761.111111  159761.111111
6     7  -9038.888889  156094.444444
7     8  -9183.333333  152422.222222
8     9  -9505.555556  148688.888889
9    10  -9500.000000  145011.111111
10   11  -9050.000000  141305.555556
11   12  -8866.666667  137583.333333
12   13  -7666.666667  133888.888889
13   14  -5516.666667  130183.333333
14   15    -38.888889  127622.222222
15   16   5455.555556  127400.000000
16   17  10950.000000  129205.555556
17   18  21022.222222  136633.333333
18   19  22216.666667  140366.666667
19   20  22833.333333  144088.888889


In [8]:
# The name of the new file to check for cell coordinates
file_to_check = 'GSM7473682_HC_a_tx_file.csv.gz'

try:
    with tarfile.open(tar_file_path, 'r') as tar:
        member = tar.getmember(file_to_check)
        file_obj = tar.extractfile(member)
        
        if file_obj is not None:
            # Read the gzipped CSV file directly into a pandas DataFrame
            df = pd.read_csv(io.BytesIO(file_obj.read()), compression='gzip')
            
            # Display the first few rows to see the column names and data
            print(f"Contents of {file_to_check}:")
            print(df.head())
        else:
            print(f"Error: Could not find '{file_to_check}' inside the archive.")
            
except Exception as e:
    print(f"An error occurred: {e}")

Contents of GSM7473682_HC_a_tx_file.csv.gz:
   fov  cell_ID   x_global_px    y_global_px  x_local_px  y_local_px  z  \
0    1        0  20357.755556  172073.335556     3202.20     2317.78 -1   
1    1        0  18652.995556  171018.275556     1497.44     1262.72 -1   
2    1        0  20339.575556  170874.485556     3184.02     1118.93 -1   
3    1        0  19901.285556  171188.505556     2745.73     1432.95 -1   
4    1        0  19507.555556  171222.455556     2352.00     1466.90 -1   

     target CellComp  
0     REG1A      NaN  
1  NegPrb18      NaN  
2      CD24      NaN  
3      GZMB      NaN  
4     REG1A      NaN  


In [9]:
# Create a list of all the files you want to check
files_to_check = [
    'GSM7473685_UC_a_tx_file.csv.gz',
    'GSM7473686_UC_b_tx_file.csv.gz',
    'GSM7473687_UC_c_tx_file.csv.gz',
    'GSM7473688_CD_a_tx_file.csv.gz',
    'GSM7473689_CD_b_tx_file.csv.gz',
    'GSM7473690_CD_c_tx_file.csv.gz'
]

try:
    with tarfile.open(tar_file_path, 'r') as tar:
        for file_name in files_to_check:
            print(f"\n--- Processing {file_name} ---")
            
            # Extract the file object from the tar archive
            member = tar.getmember(file_name)
            file_obj = tar.extractfile(member)
            
            if file_obj is not None:
                # Read the gzipped CSV directly into a pandas DataFrame
                df = pd.read_csv(io.BytesIO(file_obj.read()), compression='gzip')
                
                # Display the first 5 rows and the total number of records
                print(f"Number of records: {len(df)}")
                print(df.head())
            else:
                print(f"Error: Could not find '{file_name}' inside the archive.")
                
except Exception as e:
    print(f"An error occurred: {e}")


--- Processing GSM7473685_UC_a_tx_file.csv.gz ---
Number of records: 19424038
   fov  cell_ID  x_global_px    y_global_px  x_local_px  y_local_px  z  \
0    1        0  5027.376667 -136083.775889     1360.71    3405.113 -1   
1    1        0  5058.676667 -136558.802889     1392.01    2930.086 -1   
2    1        0  5231.516667 -136890.048889     1564.85    2598.840 -1   
3    1        0  5940.356667 -137140.108889     2273.69    2348.780 -1   
4    1        0  6810.936667 -137513.488889     3144.27    1975.400 -1   

   target  CellComp  
0   MARCO       NaN  
1   EPHB6       NaN  
2  COL3A1  Membrane  
3   HLA-B       NaN  
4    PIGR       NaN  

--- Processing GSM7473686_UC_b_tx_file.csv.gz ---
Number of records: 17542839
   fov  cell_ID   x_global_px  y_global_px  x_local_px  y_local_px  z  target  \
0    1        0 -15790.251111  -132757.289    3420.860    3492.711 -1    CCR7   
1    1        0 -18028.411111  -134520.820    1182.700    1729.180 -1    CD81   
2    1        0 -19052

In [10]:
# The name of the nested CellLabels archive inside the main archive
cell_labels_archive = 'GSM7473682_HCa_CellLabels.tar.gz'

try:
    with tarfile.open(tar_file_path, 'r') as main_tar:
        # Get the nested tar.gz file object from the main archive
        nested_file_obj = main_tar.extractfile(cell_labels_archive)
        
        if nested_file_obj is not None:
            # Open the nested tar.gz archive from the file object
            with tarfile.open(fileobj=nested_file_obj, mode='r:gz') as cell_tar:
                print(f"Contents of {cell_labels_archive}:")
                for member in cell_tar.getmembers():
                    print(f"- {member.name}")
        else:
            print(f"Error: Could not find '{cell_labels_archive}' inside the main archive.")
            
except Exception as e:
    print(f"An error occurred: {e}")

Contents of GSM7473682_HCa_CellLabels.tar.gz:
- ./GSM7473682_HCa/CellLabels
- ./GSM7473682_HCa/CellLabels/CellLabels_F007.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F013.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F008.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F004.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F012.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F011.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F016.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F005.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F006.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F002.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F010.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F015.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F001.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F014.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F003.tif
- ./GSM7473682_HCa/CellLabels/CellLabels_F009.tif


In [11]:
# The name of the nested CompartmentLabels archive inside the main archive
compartment_labels_archive = 'GSM7473682_HCa_CompartmentLabels.tar.gz'

try:
    with tarfile.open(tar_file_path, 'r') as main_tar:
        # Get the nested tar.gz file object from the main archive
        nested_file_obj = main_tar.extractfile(compartment_labels_archive)
        
        if nested_file_obj is not None:
            # Open the nested tar.gz archive from the file object
            with tarfile.open(fileobj=nested_file_obj, mode='r:gz') as compartment_tar:
                print(f"Contents of {compartment_labels_archive}:")
                for member in compartment_tar.getmembers():
                    print(f"- {member.name}")
        else:
            print(f"Error: Could not find '{compartment_labels_archive}' inside the main archive.")
            
except Exception as e:
    print(f"An error occurred: {e}")

Contents of GSM7473682_HCa_CompartmentLabels.tar.gz:
- ./GSM7473682_HCa/CompartmentLabels
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F002.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F010.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F007.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F011.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F004.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F016.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F005.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F012.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F014.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F015.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F003.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F009.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F006.tif
- ./GSM7473682_HCa/CompartmentLabels/CompartmentLabels_F013.tif
- ./GSM7473682

In [18]:
# The name of the metadata file inside the tar archive
file_to_check = 'GSM7473683_HC_b_metadata_file.csv.gz'

try:
    with tarfile.open(tar_file_path, 'r') as tar:
        # Get the member (file) from the tar archive
        member = tar.getmember(file_to_check)
        
        # Extract the file object
        file_obj = tar.extractfile(member)
        
        if file_obj is not None:
            # Read the gzipped CSV directly into a pandas DataFrame
            df = pd.read_csv(io.BytesIO(file_obj.read()), compression='gzip')
            
            print(f"Contents of {file_to_check}:")
            # Print the columns to see what information is available
            print("Columns available:", list(df.columns))
            # Display the first 5 rows to see the data format
            print(df.head())
        else:
            print(f"Error: Could not find '{file_to_check}' inside the archive.")
            
except Exception as e:
    print(f"An error occurred: {e}")

Contents of GSM7473683_HC_b_metadata_file.csv.gz:
Columns available: ['fov', 'cell_ID', 'Area', 'AspectRatio', 'CenterX_local_px', 'CenterY_local_px', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI']
   fov  cell_ID  Area  AspectRatio  CenterX_local_px  CenterY_local_px  \
0    1        1  5045         1.22              3643              3611   
1    1        2  6567         1.27              4188              3607   
2    1        3  5468         1.11              4288              3610   
3    1        4  7073         0.88              4403              3597   
4    1        5  5539         2.04              4519              3615   

   CenterX_global_px  CenterY_global_px  Width  Height  Mean.MembraneStain  \
0        6387.444444      161877.666667     90      74                6593   
1        6932.444444      161873.666667    1

In [14]:
# Define the full path to the .h5ad file
file_path = '/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_annotated_with_celltype_and_nmf_factor.h5ad'

adata = sc.read_h5ad(file_path)
    
# Print the AnnData object's summary
print("AnnData object summary:")
print(adata)
    
# Access and display the first 5 rows of the metadata (.obs)
print("\nMetadata (adata.obs.head()):")
print(adata.obs.head())

AnnData object summary:
AnnData object with n_obs × n_vars = 459095 × 980
    obs: 'cell_id', 'fov', 'patient', 'subset', 'unique_cell_id', 'PredictedCellType', 'NMF_factor'

Metadata (adata.obs.head()):
  cell_id fov patient subset unique_cell_id PredictedCellType  NMF_factor
0       1   1    CD c              CD c_1_1                N2           4
1       2   1    CD c              CD c_1_2                NK           3
2       3   1    CD c              CD c_1_3        MT T cells           4
3       4   1    CD c              CD c_1_4       Eosinophils           4
4       5   1    CD c              CD c_1_5   Cycling T cells           4


In [22]:
# --- File Paths ---
adata_path = '/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_annotated_with_celltype_and_nmf_factor.h5ad'
tar_file_path = '/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_RAW.tar'
output_path = '/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/CompleteCosMx_singlecellspatialresolution.h5ad'

# --- List of all metadata files inside the tar archive ---
metadata_files = [
    'GSM7473682_HC_a_metadata_file.csv.gz',
    'GSM7473683_HC_b_metadata_file.csv.gz',
    'GSM7473684_HC_c_metadata_file.csv.gz',
    'GSM7473685_UC_a_metadata_file.csv.gz',
    'GSM7473686_UC_b_metadata_file.csv.gz',
    'GSM7473687_UC_c_metadata_file.csv.gz',
    'GSM7473688_CD_a_metadata_file.csv.gz',
    'GSM7473689_CD_b_metadata_file.csv.gz',
    'GSM7473690_CD_c_metadata_file.csv.gz'
]

# --- Main Script ---
try:
    print("Loading existing AnnData object...")
    adata = sc.read_h5ad(adata_path)
    
    all_metadata_dfs = []
    
    print("Extracting and combining metadata files...")
    with tarfile.open(tar_file_path, 'r') as tar:
        for file_name in metadata_files:
            try:
                file_obj = tar.extractfile(file_name)
                
                if file_obj is not None:
                    df_temp = pd.read_csv(io.BytesIO(file_obj.read()), compression='gzip')
                    
                    parts = os.path.basename(file_name).split('_')
                    patient = parts[1]
                    subset = parts[2]
                    
                    df_temp['unique_cell_id'] = f'{patient} {subset}_' + df_temp['fov'].astype(str) + '_' + df_temp['cell_ID'].astype(str)
                    
                    all_metadata_dfs.append(df_temp)
                else:
                    print(f"Warning: Could not find '{file_name}' inside the archive. Skipping.")
            except KeyError:
                print(f"Warning: File {file_name} not found. Check the filename list.")
                continue
    
    if not all_metadata_dfs:
        raise ValueError("No metadata files were processed. Check your file paths and names.")
        
    combined_metadata_df = pd.concat(all_metadata_dfs, ignore_index=True)
    combined_metadata_df.set_index('unique_cell_id', inplace=True)
    
    print("\nMerging metadata into the AnnData object...")
    
    # --- NEW: Drop the overlapping 'fov' column before merging ---
    if 'fov' in adata.obs.columns:
        adata.obs.drop(columns='fov', inplace=True, errors='ignore')
    
    # Perform the merge. The .obs index should match the new metadata index.
    adata.obs = adata.obs.join(combined_metadata_df, on='unique_cell_id', how='left')
    
    # Check for successful merge (e.g., check for a new column)
    if 'CenterX_global_px' in adata.obs.columns:
        print("✅ Merge successful! Added spatial and other metadata.")
        print("\nNew AnnData metadata columns:")
        print(list(adata.obs.columns))
        
        # Save the updated AnnData object to the new file path
        print(f"\nSaving the updated AnnData object to {output_path}...")
        adata.write_h5ad(output_path)
        print("✅ File saved successfully.")
    else:
        raise ValueError("Merge failed. 'CenterX_global_px' was not added. Check for a column name mismatch or index issue.")

except FileNotFoundError as e:
    print(f"Error: {e}. One of the files could not be found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Loading existing AnnData object...
Extracting and combining metadata files...

Merging metadata into the AnnData object...
✅ Merge successful! Added spatial and other metadata.

New AnnData metadata columns:
['cell_id', 'patient', 'subset', 'unique_cell_id', 'PredictedCellType', 'NMF_factor', 'fov', 'cell_ID', 'Area', 'AspectRatio', 'CenterX_local_px', 'CenterY_local_px', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD45', 'Max.CD45', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI']

Saving the updated AnnData object to /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/CompleteCosMx_singlecellspatialresolution.h5ad...
✅ File saved successfully.


In [24]:
# Define the full path to the new, complete .h5ad file
file_path = '/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/CompleteCosMx_singlecellspatialresolution.h5ad'

# Load the AnnData object
adata = sc.read_h5ad(file_path)
    
# Access and display the first 5 rows of the metadata (.obs)
print("Metadata (adata.obs.head()):")
print(adata.obs.head())
    
# Optionally, print all columns to confirm
print("\nAll columns in adata.obs:")
print(list(adata.obs.columns))

Metadata (adata.obs.head()):
  cell_id patient subset unique_cell_id PredictedCellType  NMF_factor  fov  \
0       1    CD c              CD c_1_1                N2           4    1   
1       2    CD c              CD c_1_2                NK           3    1   
2       3    CD c              CD c_1_3        MT T cells           4    1   
3       4    CD c              CD c_1_4       Eosinophils           4    1   
4       5    CD c              CD c_1_5   Cycling T cells           4    1   

   cell_ID  Area  AspectRatio  ...  Mean.MembraneStain  Max.MembraneStain  \
0        1  6275         1.18  ...                3739              10432   
1        2  3561         0.86  ...                1835               4555   
2        3  3542         1.33  ...                7202              21030   
3        4  4798         1.01  ...                 879               3632   
4        5  4398         1.00  ...                2097               7728   

   Mean.PanCK  Max.PanCK  Mean.CD45  Ma

### Dysregulated Genes Per Disease/Health State Per Niche

In [46]:
# Path to your file
file_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/CompleteCosMx_singlecellspatialresolution.h5ad"

# Load the AnnData object
adata = ad.read_h5ad(file_path)

# Convert the cell x gene matrix (adata.X) to a pandas DataFrame
# The .X attribute is already a numpy.ndarray, so .toarray() is not needed.
cell_gene_df = pd.DataFrame(adata.X, index=adata.obs_names, columns=adata.var_names)

# Now, 'cell_gene_df' is your cell x gene matrix as a pandas DataFrame
cell_gene_df

0    1         2    3         4         5    6         7    8    9  \
0       0.0  0.0  0.000000  0.0  0.637672  0.000000  0.0  0.000000  0.0  0.0   
1       0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
2       0.0  0.0  0.000000  0.0  0.000000  0.780317  0.0  0.000000  0.0  0.0   
3       0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
4       0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
...     ...  ...       ...  ...       ...       ...  ...       ...  ...  ...   
459090  0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
459091  0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
459092  0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
459093  0.0  0.0  1.300169  0.0  0.000000  0.000000  0.0  0.000000  0.0  0.0   
459094  0.0  0.0  0.000000  0.0  0.000000  0.000000  0.0  0.850574  0.0  0.0   

              10        11   12   13   14   15        16   17        18   19  \
0       0.000000  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   
1       0.000000  0.802884  0.0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   
2       0.000000  0.780317  0.0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   
3       0.000000  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   
4       0.000000  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   
...          ...       ...  ...  ...  ...  ...       ...  ...       ...  ...   
459090  0.000000  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   
459091  0.000000  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  1.825893  0.0   
459092  0.000000  0.000000  0.0  0.0  0.0  0.0  1.396823  0.0  0.000000  0.0   
459093  0.000000  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   
459094  1.382079  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   

         20   21   22   23   24   25   26   27        28   29   30   31   32  \
0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   
1       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   
2       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   
3       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.268669  0.0  0.0  0.0  0.0   
4       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   
...     ...  ...  ...  ...  ...  ...  ...  ...       ...  ...  ...  ...  ...   
459090  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   
459091  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   
459092  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   
459093  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   
459094  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0   

         33   34        35   36        37   38        39   40   41        42  \
0       0.0  0.0  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.0  0.000000   
1       0.0  0.0  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.0  0.000000   
2       0.0  0.0  0.000000  0.0  0.780317  0.0  0.000000  0.0  0.0  0.000000   
3       0.0  0.0  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.0  1.933114   
4       0.0  0.0  0.000000  0.0  1.972025  0.0  0.000000  0.0  0.0  0.000000   
...     ...  ...       ...  ...       ...  ...       ...  ...  ...       ...   
459090  0.0  0.0  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.0  0.000000   
459091  0.0  0.0  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.0  0.000000   
459092  0.0  0.0  1.396823  0.0  0.000000  0.0  0.000000  0.0  0.0  0.000000   
459093  0.0  0.0  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.0  0.000000   
459094  0.0  0.0  0.000000  0.0  0.000000  0.0  0.850574  0.0  0.0  0.000000   

              43   44   45   46   47   48   49   50   51   52        53  \
0       0.637672  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000   
1       0.000000

In [56]:
import anndata as ad

# Path to the file
file_path = '/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_combined.h5ad'

# Load the AnnData object
try:
    adata = ad.read_h5ad(file_path)
    
    # Print a summary of the AnnData object to see its contents
    print(adata)
    
    # You can also access specific parts of the object
    print("\nObservation (cell) metadata:")
    print(adata.obs.head())
    
    print("\nVariable (gene) metadata:")
    print(adata.var.head())
    
    print("\nNumber of cells and genes:")
    print(adata.shape)
    
    # To view the actual expression matrix, you can use .X
    # For large datasets, this can consume a lot of memory, so be cautious
    # print("\nExpression matrix (first 5 rows and 5 columns):")
    # print(adata.X[:5, :5])
    
except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found. Please check the path.")

AnnData object with n_obs × n_vars = 459095 × 980
    obs: 'patient', 'cell_id', 'fov'

Observation (cell) metadata:
               patient cell_id fov
unique_cell_id                    
CD c_1_1          CD c       1   1
CD c_1_2          CD c       2   1
CD c_1_3          CD c       3   1
CD c_1_4          CD c       4   1
CD c_1_5          CD c       5   1

Variable (gene) metadata:
Empty DataFrame
Columns: []
Index: [AATK, ABL1, ABL2, ACE, ACE2]

Number of cells and genes:
(459095, 980)


In [57]:
import scanpy as sc
import pandas as pd

def correct_h5ad_ids(target_path, ref_path, output_path):
    """
    Corrects cell IDs and gene names in a target AnnData object 
    using a reference AnnData object.

    Args:
        target_path (str): Path to the AnnData file to be corrected.
        ref_path (str): Path to the reference AnnData file with correct IDs.
        output_path (str): Path to save the corrected AnnData file.
    """
    try:
        # Load the target and reference AnnData objects
        adata_target = sc.read_h5ad(target_path)
        adata_ref = sc.read_h5ad(ref_path)

        # Get unique cell IDs and gene names from the reference file
        ref_cell_ids = adata_ref.obs_names
        ref_gene_names = adata_ref.var_names
        
        # Check if the dimensions match
        if len(adata_target.obs_names) != len(ref_cell_ids):
            print("Error: The number of cells in the target and reference files do not match.")
            return
        if len(adata_target.var_names) != len(ref_gene_names):
            print("Error: The number of genes in the target and reference files do not match.")
            return

        # Assign the new unique IDs and gene names
        adata_target.obs_names = ref_cell_ids
        adata_target.var_names = ref_gene_names

        # Save the corrected AnnData object
        adata_target.write_h5ad(output_path)
        print(f"Successfully corrected and saved the file to: {output_path}")

    except FileNotFoundError as e:
        print(f"Error: One of the files was not found. Please check the paths. {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Define the file paths
target_file_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/CompleteCosMx_singlecellspatialresolution.h5ad"
ref_file_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/GSE234713_CosMx_combined.h5ad"
output_file_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Corrected_CompleteCosMx.h5ad"

# Run the function to correct the file
correct_h5ad_ids(target_file_path, ref_file_path, output_file_path)

Successfully corrected and saved the file to: /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Corrected_CompleteCosMx.h5ad


In [59]:
# Load the corrected .h5ad file
adata = sc.read_h5ad("/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Corrected_CompleteCosMx.h5ad")

# Convert the AnnData object's matrix (adata.X) to a pandas DataFrame.
# No need for .toarray() because the matrix is already a dense numpy.ndarray.
cell_by_gene_df = pd.DataFrame(adata.X, index=adata.obs_names, columns=adata.var_names)

# Print the DataFrame
cell_by_gene_df

AATK  ABL1      ABL2  ACE      ACE2     ACKR1  ACKR3  \
unique_cell_id                                                         
CD c_1_1         0.0   0.0  0.000000  0.0  0.637672  0.000000    0.0   
CD c_1_2         0.0   0.0  0.000000  0.0  0.000000  0.000000    0.0   
CD c_1_3         0.0   0.0  0.000000  0.0  0.000000  0.780317    0.0   
CD c_1_4         0.0   0.0  0.000000  0.0  0.000000  0.000000    0.0   
CD c_1_5         0.0   0.0  0.000000  0.0  0.000000  0.000000    0.0   
...              ...   ...       ...  ...       ...       ...    ...   
UC c_21_2244     0.0   0.0  0.000000  0.0  0.000000  0.000000    0.0   
UC c_21_2246     0.0   0.0  0.000000  0.0  0.000000  0.000000    0.0   
UC c_21_2247     0.0   0.0  0.000000  0.0  0.000000  0.000000    0.0   
UC c_21_2248     0.0   0.0  1.300169  0.0  0.000000  0.000000    0.0   
UC c_21_2249     0.0   0.0  0.000000  0.0  0.000000  0.000000    0.0   

                   ACKR4  ACTA2  ACTG2     ACVR1    ACVR1B  ACVR2A  ACVRL1  \
unique_cell_id                                                               
CD c_1_1        0.000000    0.0    0.0  0.000000  0.000000     0.0     0.0   
CD c_1_2        0.000000    0.0    0.0  0.000000  0.802884     0.0     0.0   
CD c_1_3        0.000000    0.0    0.0  0.000000  0.780317     0.0     0.0   
CD c_1_4        0.000000    0.0    0.0  0.000000  0.000000     0.0     0.0   
CD c_1_5        0.000000    0.0    0.0  0.000000  0.000000     0.0     0.0   
...                  ...    ...    ...       ...       ...     ...     ...   
UC c_21_2244    0.000000    0.0    0.0  0.000000  0.000000     0.0     0.0   
UC c_21_2246    0.000000    0.0    0.0  0.000000  0.000000     0.0     0.0   
UC c_21_2247    0.000000    0.0    0.0  0.000000  0.000000     0.0     0.0   
UC c_21_2248    0.000000    0.0    0.0  0.000000  0.000000     0.0     0.0   
UC c_21_2249    0.850574    0.0    0.0  1.382079  0.000000     0.0     0.0   

                ADGRA2  ADGRA3    ADGRB2  ADGRB3    ADGRD1  ADGRE1  ADGRE2  \
unique_cell_id                                                               
CD c_1_1           0.0     0.0  0.000000     0.0  0.000000     0.0     0.0   
CD c_1_2           0.0     0.0  0.000000     0.0  0.000000     0.0     0.0   
CD c_1_3           0.0     0.0  0.000000     0.0  0.000000     0.0     0.0   
CD c_1_4           0.0     0.0  0.000000     0.0  0.000000     0.0     0.0   
CD c_1_5           0.0     0.0  0.000000     0.0  0.000000     0.0     0.0   
...                ...     ...       ...     ...       ...     ...     ...   
UC c_21_2244       0.0     0.0  0.000000     0.0  0.000000     0.0     0.0   
UC c_21_2246       0.0     0.0  0.000000     0.0  1.825893     0.0     0.0   
UC c_21_2247       0.0     0.0  1.396823     0.0  0.000000     0.0     0.0   
UC c_21_2248       0.0     0.0  0.000000     0.0  0.000000     0.0     0.0   
UC c_21_2249       0.0     0.0  0.000000     0.0  0.000000     0.0     0.0   

                ADGRE5  ADGRF1  ADGRF3  ADGRF4  ADGRF5  ADGRG1  ADGRG2  \
unique_cell_id                                                           
CD c_1_1           0.0     0.0     0.0     0.0     0.0     0.0     0.0   
CD c_1_2           0.0     0.0     0.0     0.0     0.0     0.0     0.0   
CD c_1_3           0.0     0.0     0.0     0.0     0.0     0.0     0.0   
CD c_1_4           0.0     0.0     0.0     0.0     0.0     0.0     0.0   
CD c_1_5           0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...                ...     ...     ...     ...     ...     ...     ...   
UC c_21_2244       0.0     0.0     0.0     0.0     0.0     0.0     0.0   
UC c_21_2246       0.0     0.0     0.0     0.0     0.0     0.0     0.0   
UC c_21_2247       0.0     0.0     0.0     0.0     0.0     0.0     0.0   
UC c_21_2248       0.0     0.0     0.0     0.0     0.0     0.0     0.0   
UC c_21_2249       0.0     0.0     0.0     0.0     0.0     0.0     0.0   

                  ADGRG3  ADGRG5  ADGRG6  ADGRL1  ADGRL2  ADGRL4  ADGRV1  \
unique_cel

In [60]:
# --- File Paths ---
h5ad_file_path = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Corrected_CompleteCosMx.h5ad"
output_dir = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Post-NMF_Analysis/DysregulatedGenesPerDiseaseStatePerNiche"

# --- Load the AnnData object ---
print("Loading data...")
try:
    adata = sc.read(filename=h5ad_file_path)
    print("✅ AnnData object loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at {h5ad_file_path}")
    exit()

# --- Create the output directory ---
os.makedirs(output_dir, exist_ok=True)
print(f"✅ Output directory created at: {output_dir}")

# --- Prepare data for analysis ---
# Ensure 'NMF_factor' and 'patient' columns are of string type for sc.tl.rank_genes_groups
adata.obs['NMF_factor'] = adata.obs['NMF_factor'].astype(str)
adata.obs['patient'] = adata.obs['patient'].astype(str)

# Create the 'Disease_State' column from the 'patient' column
adata.obs['Disease_State'] = adata.obs['patient'].str[:2]
all_niches = sorted(adata.obs['NMF_factor'].unique())

# Define your significance thresholds
pvalue_threshold = 0.05
log2fc_threshold = 0.5

# --- Step 2: Loop and Analyze Each Niche ---
for niche in all_niches:
    print(f"\n{'='*50}\nAnalyzing Niche: {niche}\n{'='*50}")
    
    # Subset data to include only cells from the current niche
    niche_adata = adata[adata.obs['NMF_factor'] == niche, :].copy()
    
    disease_states_in_niche = niche_adata.obs['Disease_State'].unique()
    
    if len(disease_states_in_niche) < 2:
        print(f"Skipping Niche {niche}: Less than two disease states found.")
        continue
    
    # Perform the differential expression test
    sc.tl.rank_genes_groups(
        niche_adata,
        groupby='Disease_State',
        method='wilcoxon'
    )
    
    # Extract the results for all comparisons
    result = niche_adata.uns['rank_genes_groups']
    
    # Loop through each disease state to get its significant genes
    all_comparison_groups = result['names'].dtype.names
    for group_name in all_comparison_groups:
        # Create a DataFrame for this comparison
        genes = pd.DataFrame({
            'gene': result['names'][group_name],
            'pvals_adj': result['pvals_adj'][group_name],
            'log2fc': result['logfoldchanges'][group_name],
        })
        
        # --- CORRECTED: Filter for BOTH up- and down-regulated genes ---
        significant_genes = genes[
            (genes['pvals_adj'] < pvalue_threshold) & 
            (genes['log2fc'].abs() > log2fc_threshold)
        ]
        
        if not significant_genes.empty:
            print(f"\n--- Significant genes for '{group_name}' in Niche {niche} ---")
            print(significant_genes)
            
            # Save the DataFrame to a CSV file
            filename = f"Niche_{niche}_SignificantGenes_in_{group_name}.csv"
            save_path = os.path.join(output_dir, filename)
            significant_genes.to_csv(save_path, index=False)
            print(f"✅ Saved to: {save_path}")
        else:
            print(f"\n--- No significant genes found for '{group_name}' in Niche {niche} ---")

print("\n\n✅ All analyses complete and results saved.")

Loading data...
✅ AnnData object loaded successfully.
✅ Output directory created at: /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Post-NMF_Analysis/DysregulatedGenesPerDiseaseStatePerNiche

Analyzing Niche: 1

--- Significant genes for 'CD' in Niche 1 ---
         gene      pvals_adj    log2fc
0       OLFM4  1.899257e-177  1.802840
1       KRT19  9.224520e-134  0.739231
3       S100P   5.075414e-79  0.877238
4         FOS   1.041325e-70  0.751669
5    HLA-DRB1   1.147492e-57  1.056350
6       REG1A   1.189106e-44  2.687862
7        CD74   3.389693e-35  0.723549
8       ITLN1   1.447904e-33  1.094160
9        IER3   5.172443e-33  0.895107
10      KRT18   9.621279e-32  0.672661
11       KLF2   4.971923e-28  1.333800
12      IGHG1   1.266179e-23  0.836298
13      EIF5A   2.162943e-15  0.810593
14      IGHG2   3.887161e-10  0.718175
15   SERPINA1   4.670237e-10  0.753810
17   HLA-DPA1   6.500773e-09  0.827486
18      CIITA   1.318004e-08  0.835000
19     B3GNT7   1.468642e-08  0.825434
20   

In [61]:
# Define the base directory where the files are located
base_dir = "/blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Post-NMF_Analysis/DysregulatedGenesPerDiseaseStatePerNiche"

# Initialize dictionaries to store unique genes for each niche
niche_genes = {
    "Niche_1": set(),
    "Niche_2": set(),
    "Niche_3": set(),
    "Niche_4": set()
}

# List all files in the directory
all_files = os.listdir(base_dir)

# Process files for each niche
for file_name in all_files:
    if file_name.endswith(".csv"):
        file_path = os.path.join(base_dir, file_name)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Ensure the 'gene' column exists and get the unique genes
        if 'gene' in df.columns:
            genes = df['gene'].unique()
            
            # Add genes to the correct niche set
            if "Niche_1" in file_name:
                niche_genes["Niche_1"].update(genes)
            elif "Niche_2" in file_name:
                niche_genes["Niche_2"].update(genes)
            elif "Niche_3" in file_name:
                niche_genes["Niche_3"].update(genes)
            elif "Niche_4" in file_name:
                niche_genes["Niche_4"].update(genes)

# Save each set of unique genes to a separate CSV file
for niche, genes_set in niche_genes.items():
    # Convert the set to a DataFrame
    genes_df = pd.DataFrame(list(genes_set), columns=["gene"])
    
    # Define the output file name and path
    output_file_name = f"Unique_Genes_in_{niche}.csv"
    output_file_path = os.path.join(base_dir, output_file_name)
    
    # Save the DataFrame to a CSV file
    genes_df.to_csv(output_file_path, index=False)
    
    # Print a confirmation message
    print(f"Saved {len(genes_set)} unique genes for {niche} to {output_file_path}")

Saved 898 unique genes for Niche_1 to /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Post-NMF_Analysis/DysregulatedGenesPerDiseaseStatePerNiche/Unique_Genes_in_Niche_1.csv
Saved 237 unique genes for Niche_2 to /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Post-NMF_Analysis/DysregulatedGenesPerDiseaseStatePerNiche/Unique_Genes_in_Niche_2.csv
Saved 275 unique genes for Niche_3 to /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Post-NMF_Analysis/DysregulatedGenesPerDiseaseStatePerNiche/Unique_Genes_in_Niche_3.csv
Saved 406 unique genes for Niche_4 to /blue/pbenos/tan.m/IBDCosMx_scRNAseq/CosMx/Post-NMF_Analysis/DysregulatedGenesPerDiseaseStatePerNiche/Unique_Genes_in_Niche_4.csv
